# Imports

In [3]:
from fantraxapi import *
from fantraxapi.objs import *
print('imported fantrax api')
import pandas as pd
print('imported pandas')
import pickle
print('imported pickle')
import time
print('imported time')
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
print('imported selenium')
from requests import Session
print('imported requests')
import numpy as np
print('imported numpy')
import requests
print('imported requests')
from bs4 import BeautifulSoup
print('imported beautifulsoup')
from datetime import datetime, timedelta
print('imported datetime')
import warnings
print('imported warnings')
import plotly.express as px
print('imported plotly.express')
warnings.simplefilter('ignore')

imported fantrax api
imported pandas
imported pickle
imported time
imported selenium
imported requests
imported numpy
imported requests
imported beautifulsoup
imported datetime
imported warnings
imported plotly.express


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os

# Set the path to your downloaded chromedriver
CHROMEDRIVER_PATH = "./chromedriver"  # Update this path

service = Service(CHROMEDRIVER_PATH)

options = Options()
options.page_load_strategy = 'eager'
options.add_argument("--window-size=1920,1600")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.7258.128 Safari/537.36")

with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    driver.get("https://www.fantrax.com/login")
    time.sleep(120)
    pickle.dump(driver.get_cookies(), open("fantraxloggedin.cookie", "wb"))

In [ ]:
stop

# Set Current Week

In [ ]:
CURRENT_WEEK = 24
FINISHED_WEEKS = 23

# Connect to Fantrax

In [ ]:
session = Session()

with open("fantraxloggedin.cookie", "rb") as f:
    for cookie in pickle.load(f):
        session.cookies.set(cookie["name"], cookie["value"])

# league_id = "fc4ptfghlzanrcpt"
league_id = "de19pr8smd5cytbm"

api = FantraxAPI(league_id, session=session)

# Get Fantasy Matchup Data 

## Query Data

In [ ]:
matchupId = 0
matchups = pd.DataFrame()
for week, data in enumerate(api._request("getStandings", view="SCHEDULE")['tableList']):
    if week > CURRENT_WEEK + 1:
        continue
    for matchup in data['rows']:
        matchup = matchup['cells']
        tmp_df = pd.DataFrame()
        home_team = matchup[0]['content']
        home_team_id = matchup[0]['teamId']
        home_score = float(matchup[1]['content'])
        away_team = matchup[2]['content']
        away_team_id = matchup[2]['teamId']
        away_score = float(matchup[3]['content'])
        tmp_df = pd.DataFrame({'week': {0: week, 1: week}, 
                               'team': {0: home_team, 1: away_team},
                               'team_id': {0: home_team_id, 1: away_team_id},
                               'score': {0: home_score, 1: away_score},
                               'opponent': {0: away_team, 1: home_team},
                               'opponent_score': {0: away_score, 1: home_score},
                               'matchupId': {0: matchupId, 1: matchupId}
                              })
        matchups = pd.concat([matchups, tmp_df])
        matchupId += 1

## Clean Data

In [ ]:
matchups['week'] = matchups['week'].apply(lambda x: x+1)

In [ ]:
matchups = matchups.reset_index(drop=True)

In [ ]:
matchups['total_week_score'] = matchups.groupby('week')['score'].transform(lambda x: x.sum())

In [ ]:
# matchups = matchups[matchups.total_week_score > 0]
matchups = matchups[matchups.week <= CURRENT_WEEK]

In [ ]:
matchups.groupby('week').total_week_score.last().describe() 

In [ ]:
matchups['result'] = np.where(matchups.score > matchups.opponent_score, 'W', np.where(matchups.score < matchups.opponent_score, 'L', 'D'))

In [ ]:
matchups

In [ ]:
matchups[matchups.total_week_score > 0]

In [ ]:
matchups['last_score'] = matchups.groupby('team').shift(1).score.fillna(0)

In [ ]:
# total points for as of given week
matchups['cumulative_total_score'] = matchups.groupby('team').last_score.cumsum().fillna(0)

# average points for as of given week
matchups['cumulative_avg_score'] = (matchups.cumulative_total_score / (matchups.week - 1)).round(2).fillna(0)

# exponential weighted moving average points for as of given week (older scores are decayed with specified halflife)
# matchups['score_ewm'] = matchups.groupby('team').last_score.transform(lambda x: x.ewm(halflife=0.85).mean()).round(2) 

In [ ]:
matchups.set_index(['week', 'team']).tail(28).head(14)

## Analyze Data

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# from plotly.offline import init_notebook_mode, iplot
# init_notebook_mode(connected=True)

In [ ]:
matchups['results_points'] = np.where(matchups.result == 'W', 3, 
                             np.where(matchups.result == 'D', 1,
                                                              0))

In [ ]:
matchups['table_points'] = matchups.groupby('team').results_points.expanding().sum().reset_index().set_index('level_1').sort_index().results_points

In [ ]:
completed_matchups = matchups[matchups.total_week_score > 0]

In [ ]:
px.line(data_frame=completed_matchups, x='week', y='score', color='team')

In [ ]:
px.line(data_frame=completed_matchups, x='week', y='table_points', color='team')

## Calculate Standings

In [ ]:
standings = pd.DataFrame()
standings['team_id'] = completed_matchups.groupby('team').team_id.last()
standings['wins'] = completed_matchups.groupby('team').result.agg(lambda x: x.tolist().count("W"))
standings['draws'] = completed_matchups.groupby('team').result.agg(lambda x: x.tolist().count("D"))
standings['losses'] = completed_matchups.groupby('team').result.agg(lambda x: x.tolist().count("L"))
standings['record'] = standings.wins.astype(str) + '-' + standings.draws.astype(str) + '-' + standings.losses.astype(str)
standings['points'] = standings.wins*3 + standings.draws
standings['fpts_for'] = completed_matchups.groupby('team').score.sum()
standings['fpts_against'] = completed_matchups.groupby('team').opponent_score.sum()
standings['avg_fpts_for'] = completed_matchups.groupby('team').score.mean().round(2)
standings['avg_fpts_against'] = completed_matchups.groupby('team').opponent_score.mean().round(2)
standings['std_fpts_for'] = completed_matchups.groupby('team').score.std().round(2)

In [ ]:
standings.sort_values(['points', 'fpts_for'], ascending=False)[['record', 'points', 'fpts_for', 'fpts_against', 'avg_fpts_for', 'avg_fpts_against']]

## Attach Fantasy Lineups

In [ ]:
# weeks = standings[['wins', 'draws', 'losses']].sum(axis=1).max()
weeks = CURRENT_WEEK
weeks

In [ ]:
# weeks=weeks+1

In [ ]:
teams = api.teams

In [ ]:
data = api._request("getTeamRosterInfo")

In [ ]:
season_data = pd.DataFrame()
x = 0
for week in range(1, weeks+1):
    print(week, end=': ')
    for team_name in api.teams:
        player_row = {}
        player_row['week'] = {x: week}
        player_row['team'] = {x: team_name}
        team_name = str(team_name)
        team_id = standings.loc[team_name, 'team_id']
        roster = Roster(api, api._request("getTeamRosterInfo", period=week, teamId=team_id, timeStartType="FROM_SEASON_START"), team_id).rows
        sorted_roster = []
        reserves = []
        injured_reserve = []
        for i, row in enumerate(roster):
            if row.pos.short_name == 'Res':
                reserves += [row]
            elif row.pos.short_name == 'IR':
                injured_reserve += [row]
            else:
                sorted_roster += [row]
        sorted_roster += reserves + injured_reserve
        for i, row in enumerate(sorted_roster):
            player_row[i] = {x: row.player}
        player_row = pd.DataFrame(player_row)
        season_data = pd.concat([season_data, player_row])
    x+=1
    print(season_data.shape)

In [ ]:
season_data = season_data.reset_index(drop=True)

In [ ]:

season_data.columns = ['week', 'team', 'g'] + [f'd{i}' for i in range(1, 6)] + [f'm{i}' for i in range(1, 6)] + [f'f{i}' for i in range(1, 4)] + [f'res{i}' for i in range(1, 6)] + ['inj']

In [ ]:
season_data['team'] = season_data['team'].apply(lambda x: str(x))

In [ ]:
matchup_data = pd.concat([matchups.sort_values(['week', 'team']).reset_index(drop=True), season_data.sort_values(['week', 'team']).reset_index(drop=True)], axis=1)

In [ ]:
matchup_data = matchup_data.loc[:, ~matchup_data.columns.duplicated()]

# Get Player Data

## Query Data

In [ ]:
response = api._request('getPlayerStats', statusOrTeamFilter='ALL', pageNumber=i)

In [ ]:
df_players = pd.DataFrame()
for i in range(1, api._request('getPlayerStats', statusOrTeamFilter='ALL', pageNumber=1)['paginatedResultSet']['totalNumPages']+1):
    i = str(i)
    response = api._request('getPlayerStats', statusOrTeamFilter='ALL', pageNumber=i)
    for player_data in response['statsTable']:
        data = player_data['scorer']
        data.pop('icons', None)
        cells = player_data['cells']
        data['next_match'] = cells[2]['content']
        data['fpts'] = cells[3]['content']
        data['fpts_per_game'] = cells[4]['content']
        # data['rostered'] = cells[7]['content']
        # data['trend'] = cells[8]['content']
        tmp_df = pd.DataFrame(data)
        df_players = pd.concat([df_players, tmp_df]).reset_index(drop=True)

## Clean Data

In [ ]:
df_players['next_match'] = df_players.next_match.apply(lambda x: x.replace('<br/>', ' '))

In [ ]:
df_players['opponentShortName'] = df_players.next_match.apply(lambda x: x.replace('@', '')[:3])

In [ ]:
team_dict = df_players[['teamShortName', 'teamName']].drop_duplicates().set_index('teamShortName')

In [ ]:
team_dict = pd.concat([team_dict, pd.DataFrame({'teamShortName': {0: ''}, 'teamName': {0: np.nan}}).set_index('teamShortName')])

In [ ]:
df_players['opponentName'] = df_players.opponentShortName.apply(lambda x: team_dict.loc[x]['teamName'])

In [ ]:
# df_players['trend'] = df_players.trend.fillna(0).apply(lambda x: float(x.replace('%', '')))

In [ ]:
df_players[['name', 'scorerId', 'teamShortName', 'teamName', 'posShortNames', 'opponentShortName', 'opponentName', 'fpts', 'fpts_per_game']].head(5)

# Get Player Performance Data

## Util

In [ ]:
# Function to assign the correct year based on the season logic
def add_season_year(date_str, start_year):
    # Parse the date without year
    parsed_date = datetime.strptime(date_str, '%b %d')
    
    # Determine the year based on the season (Aug–May)
    if parsed_date.month >= 8:  # August to December -> 2024
        year = start_year
    else:  # January to May -> 2025
        year = start_year + 1
    
    # Construct the full date
    full_date = datetime(year, parsed_date.month, parsed_date.day)
    return full_date

def get_player_data(playerId, year=2025):
    json_data = {
        "msgs": [
            {
                "method": "getPlayerProfile",
                "data": {
                    "playerId": playerId,
                    "tab": "GAME_LOG_FANTASY",
                    "seasonId": str(year - 1100)
                }
            }
        ],
        "uiv": 3,
        "refUrl": "https://www.fantrax.com/fantasy/league/fc4ptfghlzanrcpt/players;reload=1",
        "dt": 1,
        "at": 0,
        "av": "0.0",
        "tz": "America/New_York",
        "v": "179.0.1"
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
    }

    r = api._session.post(f"https://www.fantrax.com/fxpa/req?leagueId={api.league_id}", json=json_data, headers=headers)
    p = r.json()['responses'][0]['data']['sectionContent']['GAME_LOG_FANTASY']['tables'][0]['rows']
    columns = [cell['shortName'] for cell in r.json()['responses'][0]['data']['sectionContent']['GAME_LOG_FANTASY']['tables'][0]['header']['cells']]
    df = pd.DataFrame(columns=columns)
    for week in range(0, 38+1):
        try:
            tmp_df = pd.DataFrame(p[week]['cells'])[['content']].T
            tmp_df.columns = columns
            df = pd.concat([df, tmp_df])
        except Exception as e:
            pass
    df = pd.concat([append_next_week(playerId), df])
    df['player_id'] = playerId
    df['player_name'] = df_players[df_players.scorerId==playerId].name.tolist()[0]
    df = df.reset_index(drop=True).sort_index(ascending=False).reset_index(drop=True)
    season = year
    df['Date'] = df['Date'].apply(lambda x: add_season_year(x, season))
    return df.set_index('Date')


def append_next_week(playerId):
    urlName = df_players[(df_players.scorerId == playerId)]['urlName'].values[0]
    json_data = {
        "msgs": [
            {
                "method": "getPlayerProfile",
                "data": {
                    "playerId": playerId,
                }
            }
        ],
        "uiv": 3,
        "refUrl": f"https://www.fantrax.com/player/{playerId}/{league_id}/{urlName}",
        "dt": 1,
        "at": 0,
        "av": "0.0",
        "tz": "America/New_York",
        "v": "179.0.1"
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
    }
    
    r = api._session.post(f"https://www.fantrax.com/fxpa/req?leagueId={api.league_id}", json=json_data, headers=headers)
    p = r.json()['responses'][0]['data']['sectionContent']['OVERVIEW']['tables'][3]['rows'][0]['cells']
    team = df_players[df_players.scorerId == playerId].teamShortName.values[0]
    opponent_raw = p[1]['content'].split(' ')[0]
    opponent = df_players[df_players.teamName.str.contains(opponent_raw.replace('@', ''))]['teamShortName'].iloc[0]
    if opponent_raw[0] == '@':
        opponent = '@'+opponent
    date = (p[0]['content'].split(' ')[1] + ' ' + p[0]['content'].split(' ')[2])[:-1]
    return pd.DataFrame({
        'Date': {CURRENT_WEEK: date}, 
        'Team': {CURRENT_WEEK: team},
        'Opp': {CURRENT_WEEK: opponent},
        'Score': {CURRENT_WEEK: 'L 0-0'}
    })

def get_latest_news(playerId):
    urlName = df_players[(df_players.scorerId == playerId)]['urlName'].values[0]
    print(urlName)
    json_data = {
        "msgs": [
            {
                "method": "getPlayerProfile",
                "data": {
                    "playerId": playerId,
                }
            }
        ],
        "uiv": 3,
        "refUrl": f"https://www.fantrax.com/player/{playerId}/fc4ptfghlzanrcpt/{urlName}",
        "dt": 1,
        "at": 0,
        "av": "0.0",
        "tz": "America/New_York",
        "v": "179.0.1"
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
    }
    
    r = api._session.post(f"https://www.fantrax.com/fxpa/req?leagueId={api.league_id}", json=json_data, headers=headers).json()['responses'][0]['data']['sectionContent']['OVERVIEW']
    if 'latestNews' in r.keys():
        return r['latestNews']['analysisText']
    else:
        return ''

In [ ]:
playerId = '06ex4'
urlName = df_players[(df_players.scorerId == playerId)]['urlName'].values[0]
json_data = {
    "msgs": [
        {
            "method": "getPlayerProfile",
            "data": {
                "playerId": playerId,
            }
        }
    ],
    "uiv": 3,
    "refUrl": f"https://www.fantrax.com/player/{playerId}/{league_id}/{urlName}",
    "dt": 1,
    "at": 0,
    "av": "0.0",
    "tz": "America/New_York",
    "v": "179.0.1"
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
}


r = api._session.post(f"https://www.fantrax.com/fxpa/req?leagueId={api.league_id}", json=json_data, headers=headers)
p = r.json()['responses'][0]['data']['sectionContent']['OVERVIEW']['tables'][3]['rows'][0]['cells']
team = df_players[df_players.scorerId == playerId].teamShortName.values[0]
opponent_raw = p[1]['content'].split(' ')[0]
opponent = df_players[df_players.teamName.str.contains(opponent_raw.replace('@', ''))]['teamShortName'].iloc[0]
if opponent_raw[0] == '@':
    opponent = '@'+opponent
date = (p[0]['content'].split(' ')[1] + ' ' + p[0]['content'].split(' ')[2])[:-1]

p = r.json()['responses'][0]['data']['sectionContent']['OVERVIEW']
injury_status = 'Available'
if 'injuryInfo' in p.keys():
    injury_info = p['injuryInfo']['injuryMsgs'][0]
    if 'out for next game' in injury_info.lower():
        injury_status = 'Out'
    elif 'game-time decision' in injury_info.lower():
        injury_status = 'GTD'
    else:
        pass
    
a = pd.DataFrame({
    'Date': {CURRENT_WEEK: date}, 
    'Team': {CURRENT_WEEK: team},
    'Opp': {CURRENT_WEEK: opponent},
    'Score': {CURRENT_WEEK: 'L 0-0'},
    'Injured': {CURRENT_WEEK: injury_status}
})

In [ ]:
a

In [ ]:
r.json()['responses'][0]['data']['sectionContent']['OVERVIEW']['injuryInfo']['injuryMsgs'][0]

In [ ]:
stop

## Query Data

In [ ]:
import pandas as pd

In [ ]:
df_player_stats_past_seasons = pd.DataFrame()
# start = 2022
end = 2025
# for year in range(start, end):
    # print('-'*100)
    # print('-'*47, year, '-'*47)
    # print('-'*100)
    # df_player_stats_season = pd.DataFrame()
    # for player in df_players.scorerId:
    #     try:
    #         player_data = get_player_data(player, year)
    #         print(player_data.player_name.tolist()[0], player_data.shape)
    #         df_player_stats_season = pd.concat([df_player_stats_season, player_data])
    #     except Exception as e:
    #         pass
    # df_player_stats_season = df_player_stats_season.dropna(subset='FPts')
    # df_player_stats_season = year
    # df_player_stats_season.to_csv(f'player_stats_{year}.csv')
    # df_player_stats_past_seasons = pd.concat([df_player_stats_past_seasons, pd.read_csv(f'player_stats_{year}.csv')])

df_player_stats_season = pd.DataFrame()
for player in df_players.scorerId:
    try:
        player_data = get_player_data(player, end)
        print(player_data.player_name.tolist()[0], player_data.shape)
        df_player_stats_season = pd.concat([df_player_stats_season, player_data])
    except Exception as e:
        pass
df_player_stats_season.dropna(subset='FPts')
df_player_stats_season['season'] = end
df_player_stats_season.reset_index(inplace=True)

In [ ]:
df_player_stats_season.columns

In [ ]:
# df_player_stats = pd.concat([df_player_stats_past_seasons.reset_index(), df_player_stats_season]).drop(['Unnamed: 0', 'index'], axis=1)

In [ ]:
df_player_stats = df_player_stats_season

In [ ]:
df_player_stats['Date'] = pd.to_datetime(df_player_stats.Date)

In [ ]:
raw_df_player_stats = df_player_stats.copy() # TO AVOID RERUNNING THE ABOVE

In [ ]:
df_player_stats = raw_df_player_stats.copy()

In [ ]:
# df_player_stats = df_player_stats[~(df_player_stats.Score == 'L 0-0')]
# df_player_stats.shape

## Clean Data

In [ ]:
df_player_stats.GAO.fillna(df_player_stats.GA, inplace=True)

In [ ]:
df_player_stats.columns = [col.lower() for col in df_player_stats.columns]

In [ ]:
team_games = (
    df_player_stats[['team', 'date']]
    .drop_duplicates()
    .sort_values(['team', 'date'])
    .reset_index(drop=True)
)

team_games['matchweek'] = (
    team_games
        .groupby('team')
        .cumcount()
        .add(1)
)

In [ ]:
df_player_stats = df_player_stats.merge(
    team_games,
    on=['team', 'date'],
    how='left'
)

In [ ]:
df_player_stats

In [ ]:
# Fix matchweek for ARS v BRE 2025-01-01
df_player_stats.loc[(df_player_stats.team.isin(['ARS', 'BRF']) & (df_player_stats.date == pd.Timestamp('2025-01-01'))), 'matchweek'] = 19
df_player_stats['matchweek'] = np.where((df_player_stats.matchweek > 19) & (df_player_stats.season == 2024), df_player_stats.matchweek - 1, df_player_stats.matchweek)

# Fix matchweek for LIV v EVE 2025-02-12
df_player_stats.loc[(df_player_stats.team.isin(['LIV', 'EVE']) & (df_player_stats.date == pd.Timestamp('2025-02-12'))), 'matchweek'] = 24
df_player_stats['matchweek'] = np.where((df_player_stats.matchweek > 25) & (df_player_stats.season == 2024), df_player_stats.matchweek - 1, df_player_stats.matchweek)

# Fix matchweek for LIV v AVL 2025-02-19
df_player_stats.loc[(df_player_stats.team.isin(['LIV', 'AVL']) & (df_player_stats.date == pd.Timestamp('2025-02-19'))), 'matchweek'] = 25
df_player_stats['matchweek'] = np.where((df_player_stats.matchweek > 25) & (df_player_stats.season == 2024), df_player_stats.matchweek - 1, df_player_stats.matchweek)

# Update postponed matches

In [ ]:
df_player_stats['home_ind'] = df_player_stats.opp.str.contains('@')

In [ ]:
df_player_stats['opp'] = df_player_stats.opp.apply(lambda x: x.replace('@', ''))

In [ ]:
df_player_stats.fillna(0, inplace=True)

In [ ]:
df_player_stats[df_player_stats.player_name.str.lower().str.contains('lamm')]

# Calculate Features

## Util

In [ ]:
fantasy_scoring_rules_goalkeeper = {
    "AER": 1,     # Aerials Won
    "AT": 7,      # Assists (Total)
    "CS": 8,      # Clean Sheets On Field
    "DIS": -0.5,  # Dispossessed
    "GAO": -2,     # Goals Against
    "G": 10,      # Goals
    "HCS": 1,     # High Claims Succeeded
    "Int": 1,     # Interceptions
    "OG": -5,     # Own Goals
    "PKS": 8,     # Penalty Kick Saves
    "RC": -7,     # Red Cards
    "SOT": 2,     # Shots on Target
    "Sm": 1,      # Smothers
    "CoS": 1,     # Successful Dribbles (Contests Succeeded)
    "CLR": 0.25,  # Effective Clearances
    "KP": 2,      # Key Passes (Assists on Shots)
    "Sv": 2,      # Saves
    "TkW": 1,     # Tackles Won
    "YC": -2      # Yellow Cards
}

fantasy_scoring_rules_defender = {
    "ACNC": 1,   # Accurate Crosses (No Corners)
    "AER": 1,    # Aerials Won
    "AT": 7,     # Assists (Total)
    "BS": 1,     # Blocked Shots
    "CS": 6,     # Clean Sheets On Field
    "DIS": -0.5, # Dispossessed
    "GAO": -2,   # Goals Against Outfielders
    "G": 10,     # Goals
    "Int": 1,    # Interceptions
    "OG": -5,    # Own Goals
    "PKD": 2,    # Penalty Kicks Drawn
    "PKM": -4,   # Penalty Kicks Missed
    "RC": -7,    # Red Cards
    "SOT": 2,    # Shots on Target
    "CoS": 1,    # Successful Dribbles (Contests Succeeded)
    "CLR": 0.25, # Effective Clearances
    "KP": 2,     # Key Passes (Assists on Shots)
    "TkW": 1,    # Tackles Won
    "YC": -2     # Yellow Cards
}

fantasy_scoring_rules_midfield = {
    "ACNC": 1,   # Accurate Crosses (No Corners)
    "AER": 0.5,  # Aerials Won
    "AT": 6,     # Assists (Total)
    "BS": 1,     # Blocked Shots
    "CS": 1,     # Clean Sheets On Field
    "DIS": -0.5, # Dispossessed
    "G": 9,      # Goals
    "Int": 1,    # Interceptions
    "OG": -5,    # Own Goals
    "PKD": 2,    # Penalty Kicks Drawn
    "PKM": -4,   # Penalty Kicks Missed
    "RC": -7,    # Red Cards
    "SOT": 2,    # Shots on Target
    "CoS": 1,    # Successful Dribbles (Contests Succeeded)
    "KP": 2,     # Key Passes (Assists on Shots)
    "TkW": 1,    # Tackles Won
    "YC": -2     # Yellow Cards
}

fantasy_scoring_rules_forward = {
    "ACNC": 1,   # Accurate Crosses (No Corners)
    "AER": 0.5,  # Aerials Won
    "AT": 6,     # Assists (Total)
    "BS": 1,     # Blocked Shots
    "DIS": -0.5, # Dispossessed
    "G": 9,      # Goals
    "Int": 1,    # Interceptions
    "OG": -5,    # Own Goals
    "PKD": 2,    # Penalty Kicks Drawn
    "PKM": -4,   # Penalty Kicks Missed
    "RC": -7,    # Red Cards
    "SOT": 2,    # Shots on Target
    "CoS": 1,    # Successful Dribbles (Contests Succeeded)
    "KP": 2,     # Key Passes (Assists on Shots)
    "TkW": 1,    # Tackles Won
    "YC": -2     # Yellow Cards
}

fantasy_scoring_rules = pd.DataFrame({'G':fantasy_scoring_rules_goalkeeper, 'D': fantasy_scoring_rules_defender, 'M': fantasy_scoring_rules_midfield, 'F': fantasy_scoring_rules_forward}).fillna(0).T

In [ ]:
def get_fantasy_score(row):
    stats = fantasy_scoring_rules.columns.tolist()
    score = 0
    for stat in stats:
        if stat.lower() in row.index.tolist():
            score += float(fantasy_scoring_rules.loc[row.position, stat]) * float(row[stat.lower()])
    return score

def get_adjusted_fantasy_score(row):
    stats = fantasy_scoring_rules.columns.tolist()
    score = 0
    for stat in stats:
        if stat in ['G', 'A'] or (row.position == 'D' and stat == 'CS'):
            continue
        if stat.lower() in row.index.tolist():
            score += float(fantasy_scoring_rules.loc[row.position, stat]) * float(row[stat.lower()])
    return score

In [ ]:
df_player_stats = pd.merge(left=df_player_stats, right=df_players[['scorerId', 'posShortNames']], left_on='player_id', right_on='scorerId', validate='m:1').rename(columns={'posShortNames':'position'})

In [ ]:
df_player_stats['fpts'] = df_player_stats['fpts'].astype(float)

In [ ]:
df_player_stats['calc_fpts'] = df_player_stats.apply(get_fantasy_score, axis=1)

In [ ]:
df_player_stats['apts'] = df_player_stats.apply(get_adjusted_fantasy_score, axis=1)

In [ ]:
# Not exactly 100% due to late stat corrections

In [ ]:
(df_player_stats.calc_fpts == df_player_stats.fpts).sum() / len(df_player_stats) * 100

## Fantasy Performance Based Features

In [ ]:
df_player_stats['minutes'] = df_player_stats['min'].astype(int)
# df_player_stats = df_player_stats[(df_player_stats.minutes >= 1) | (df_player_stats.matchweek == CURRENT_WEEK)]

In [ ]:
df_player_stats = df_player_stats.sort_values(['season', 'matchweek', 'player_id'], ascending=[True, True, False]).reset_index(drop=True)

In [ ]:
df_player_stats

In [ ]:
calc_stats = ['fpts', 'apts', 'g', 'kp', 'at', 'sot', 'tkw', 'dis', 'acnc', 'int', 'clr', 'cos', 'bs', 'aer', 'gao', 'cs', 'sv', 'pks', 'hcs', 'sm']
df_player_stats[f'last_minutes'] = df_player_stats.groupby(['season', 'player_id'])['minutes'].shift(1).fillna(0)
df_player_stats[f'season_minutes'] = df_player_stats.groupby(['season', 'player_id'])['last_minutes'].cumsum()

for stat in calc_stats:
    print(stat)
    if not stat in ['fpts', 'apts']:
        df_player_stats[stat] = df_player_stats[stat].fillna(0).astype(int)
    df_player_stats[f'last_{stat}'] = df_player_stats.groupby(['season', 'player_id'])[f'{stat}'].shift(1).fillna(0)
    df_player_stats[f'season_{stat}'] = df_player_stats.groupby(['season', 'player_id'])[f'last_{stat}'].cumsum()
    df_player_stats[f'{stat}_per_min_gameweek'] = (df_player_stats[f'last_{stat}'].fillna(0).astype(float) / df_player_stats.last_minutes.fillna(0).astype(int)).round(3)
    df_player_stats[f'{stat}_per_min_season'] = (df_player_stats[f'season_{stat}'].fillna(0).astype(float) / df_player_stats.season_minutes.fillna(0).astype(int)).round(3)
    df_player_stats[f'{stat}_per_90_gameweek'] = df_player_stats[f'{stat}_per_min_gameweek'] * 90
    df_player_stats[f'{stat}_per_90_season'] = df_player_stats[f'{stat}_per_min_season'] * 90

In [ ]:
df_player_stats

### Merge Fantrax Projections

In [ ]:
projections = pd.DataFrame()
for i in range(1, weeks+1):
    json_data = {
        "msgs": [
            {
                "method": "getLiveScoringStats",
                "data": {
                    "sppId": "-1",
                    "teamId": "ALL",
                    "period": i,
                    "viewType": "2",
                    "playerViewType": "2",
                    "newView": False
                }
            },
            {
                "method": "getScoresSummaryData",
                "data": {}
            }
        ],
        "uiv": 3,
        "refUrl": "https://www.fantrax.com/fantasy/league/fc4ptfghlzanrcpt/livescoring;period=12?mobileMatchupView=true&matchupId=wmhmc3gklzbqk4rv_9hdr1i7ulzfvpebf&layout=DETAILED&mainView=MATCHUP&mobileView=MATCHUP&teamId=ALL&mode=fpts",
        "dt": 1,
        "at": 0,
        "av": "0.0",
        "tz": "America/New_York",
        "v": "179.0.1"
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
    }
    
    r = api._session.post(f"https://www.fantrax.com/fxpa/req?leagueId={api.league_id}", json=json_data, headers=headers).json()['responses'][0]['data']['statsPerTeam']['allTeamsStats']
    for team in r:
        for players in ['ACTIVE', 'BENCH']:
            tmp_df = pd.DataFrame(pd.Series(r[team][players]['projectedTotalsMap'])).reset_index().rename(columns={'index': 'scorerId', 0: 'fantrax_projection'})
            tmp_df['week'] = i
            projections = pd.concat([projections, tmp_df])

In [ ]:
df_player_stats = pd.merge(df_player_stats, projections, left_on=['scorerId', 'matchweek'], right_on=['scorerId', 'week'], how='left').drop('week', axis=1)

In [ ]:
df_player_stats[(df_player_stats.season == df_player_stats.season.max()) & (df_player_stats.matchweek == CURRENT_WEEK)]

## League Table Based Features

In [ ]:
per90_cols = [c for c in df_player_stats.columns if 'per_90' in c]

def trimmed_mean(s):
    return s[s.between(s.quantile(0.05), s.quantile(0.95))].mean()

df_positional_avg_stats_against_opp = (
    df_player_stats
    .groupby(['opp', 'position'])[per90_cols]
    .apply(lambda g: g.apply(trimmed_mean))  # apply column-wise within each group
    .reset_index()
    .rename(columns={col: f'positional_avg_{col}' for col in per90_cols})
)

In [ ]:
df_player_stats = pd.merge(left=df_player_stats, right=df_positional_avg_stats_against_opp, on=['opp', 'position'])

In [ ]:
df_player_stats

In [ ]:
df_player_stats['last_score'] = df_player_stats.groupby('player_id').score.shift(1).fillna('L 0-0')
df_player_stats['team_result'] = np.where(df_player_stats['last_score'].str[0] == 'W', 3,
                                 np.where(df_player_stats['last_score'].str[0] == 'T', 1, 0))

In [ ]:
from statistics import mode
df = df_player_stats.groupby(['season', 'team', 'matchweek']).team_result.agg(lambda x: mode(list(x))).groupby(['season', 'team']).cumsum().reset_index().rename(columns={'team_result': 'table_points'})

In [ ]:
df[(df.season == 2023) & (df.matchweek == 41)].sort_values('table_points', ascending=False)

In [ ]:
df_player_stats = pd.merge(left=df_player_stats, right=df, on=['team', 'matchweek'])

In [ ]:
df_player_stats['last_score'] = df_player_stats.groupby('player_id').score.shift(1).fillna('L 0-0')
df_player_stats['last_week_goals_for'] = df_player_stats.last_score.apply(lambda x: int(x[2:].split('-')[0]))
df_player_stats['last_week_goals_against'] = df_player_stats.last_score.apply(lambda x: int(x[2:].split('-')[1]))

In [ ]:
df_player_stats['total_team_goals_for'] = df_player_stats.groupby('player_name').last_week_goals_for.cumsum()
df_player_stats['total_team_goals_against'] = df_player_stats.groupby('player_name').last_week_goals_against.cumsum()
df_player_stats['goal_differential'] = df_player_stats["total_team_goals_for"] - df_player_stats["total_team_goals_against"]

In [ ]:
df_player_stats

In [ ]:
# # FILL MISSING MATCHES, THIS IS IMPORTANT
# i = df_player_stats.index[-1] + 1
# missing_matches = {
# #  Matchweek       Teams that missed games
#           15:      ['LIV', 'EVE']
# }
# for week, teams in missing_matches.items():
#     for team in teams:
#         tmp_df = pd.DataFrame({'matchweek': {i: week}, 'team': {i: team}})
#         df_player_stats = pd.concat([df_player_stats, tmp_df])
#         for col in ["table_points", "goal_differential", "total_team_goals_for", "total_team_goals_against"]:
#             df_player_stats[col] = df_player_stats.groupby('team')[col].ffill()

# # UNDO
# # df_player_stats = df_player_stats.iloc[:i]

In [ ]:
standings = (
    df_player_stats.groupby(["matchweek", "team"], as_index=False)
    .first()
    .sort_values(by=["matchweek", "table_points", "goal_differential", "total_team_goals_for", "total_team_goals_against"], ascending=[True, False, False, False, True])
)[['matchweek', 'team', 'table_points']]

In [ ]:
standings['team_week_standing'] = standings.groupby('matchweek').cumcount().add(1).rename("team_week_rank")

In [ ]:
standings = standings.sort_values(['matchweek', 'team_week_standing', 'table_points']).reset_index(drop=True)

In [ ]:
standings[standings.matchweek == CURRENT_WEEK].set_index('team_week_standing')

In [ ]:
df_player_stats = pd.merge(left=df_player_stats, right=standings, on=['matchweek', 'team'])
df_player_stats = pd.merge(left=df_player_stats, right=standings, left_on=['matchweek', 'opp'], right_on=['matchweek', 'team'], suffixes=[None, '_opp']).drop('team_opp', axis=1)

# Calculate Manager Stats

## Util

In [ ]:
df_player_stats.matchweek.value_counts()

In [ ]:
def get_max_points_for(row):
    print('#'*40, f' Week {row.week}: {row.team}', '#'*40)
    week = row.week
    goalies = [str(row.g)]
    defenders = [str(player) for player in [row.d1, row.d2, row.d3, row.d4, row.d5] if player]
    midfielders = [str(player) for player in [row.m1, row.m2, row.m3, row.m4, row.m5] if player]
    forwards = [str(player) for player in [row.f1, row.f2, row.f3] if player]
    reserves = [str(player) for player in [row.res1, row.res2, row.res3, row.res4, row.res5, row.inj] if player]
    for res in reserves:
        try:
            res_pos = df_players[df_players.name == res].posShortNames.iloc[0]
            if res_pos == 'G':
                goalies += [res]
            elif res_pos == 'D':
                defenders += [res]
            elif res_pos == 'M':
                midfielders += [res]
            elif res_pos == 'F':
                forwards += [res]
        except:
            print(f'{row.team} is missing a reserve player')

    print('G:', goalies)
    print('D:', defenders)
    print('M:', midfielders)
    print('F:', forwards)
    
    def get_player_score(name, week):
        try:
            return df_player_stats.loc[(df_player_stats.matchweek == week) & (df_player_stats.player_name == name)].fpts.values[0]
        except:
            return 0

    g_scores = sorted([get_player_score(g, week) for g in goalies])
    d_scores = sorted([get_player_score(d, week) for d in defenders])
    m_scores = sorted([get_player_score(m, week) for m in midfielders])
    f_scores = sorted([get_player_score(f, week) for f in forwards])

    print('~'*25)
    print('G:', g_scores)
    print('D:', d_scores)
    print('M:', m_scores)
    print('F:', f_scores)
    print('~'*25)

    top_g_scores = g_scores[-1:]
    top_d_scores = d_scores[-3:]
    top_m_scores = m_scores[-2:]
    top_f_scores = f_scores[-1:]

    d_scores = d_scores[:-3]
    m_scores = m_scores[:-2]
    f_scores = f_scores[:-1]
    g_scores = g_scores[:-1]
    
    for _ in range(4):
        print('-'*50)
        print('--- TOP SCORERS ---')
        print('G:', top_g_scores)
        print('D:', top_d_scores)
        print('M:', top_m_scores)
        print('F:', top_f_scores)
        print(sum(top_g_scores), '+',
              sum(top_d_scores), '+', 
              sum(top_m_scores), '+', 
              sum(top_f_scores), '=', 
              sum(top_g_scores + top_d_scores + top_m_scores + top_f_scores))
        print()
        print('--- REMAINING PLAYERS ---')
        print('G:', g_scores)
        print('D:', d_scores)
        print('M:', m_scores)
        print('F:', f_scores)
        print()
        check_d = len(d_scores) > 0 and len(top_d_scores) <= 5
        check_m = len(m_scores) > 0 and len(top_m_scores) <= 5
        check_f = len(f_scores) > 0 and len(top_f_scores) <= 3
        if 1 + len(top_d_scores) + len(top_m_scores) + len(top_f_scores) == 11:
            break
        
        print(check_d, check_m, check_f)
        scores = []
        if check_d:
            scores += [d_scores[-1]]
        if check_m:
            scores += [m_scores[-1]]
        if check_f:
            scores += [f_scores[-1]]  
        print(scores)
        print()
        if not scores:
            break
        if check_d and max(scores) == d_scores[-1]:
            top_d_scores += [d_scores[-1]]
            d_scores = d_scores[:-1]
            continue
        if check_m and max(scores) == m_scores[-1]:
            top_m_scores += [m_scores[-1]]
            m_scores = m_scores[:-1]
            continue
        if check_f and max(scores) == f_scores[-1]:
            top_f_scores += [f_scores[-1]]
            f_scores = f_scores[:-1]
            continue

    print(sum(top_g_scores), '+',
          sum(top_d_scores), '+', 
          sum(top_m_scores), '+', 
          sum(top_f_scores), '=', 
          sum(top_g_scores + top_d_scores + top_m_scores + top_f_scores))
    print()
    print()
        
    return sum(top_g_scores + top_d_scores + top_m_scores + top_f_scores)

In [ ]:
completed_matchup_data = matchup_data[matchup_data.week <= FINISHED_WEEKS]

In [ ]:
completed_matchup_data

## Calculate Max Points For

In [ ]:
completed_matchup_data['max_points_for'] = completed_matchup_data.apply(get_max_points_for, axis=1)

In [ ]:
completed_matchup_data[['week', 'team', 'score', 'max_points_for']]

In [ ]:
completed_matchup_data['adjusted_result'] = np.where(completed_matchup_data.max_points_for > completed_matchup_data.opponent_score, 'W',
                                  np.where(completed_matchup_data.max_points_for < completed_matchup_data.opponent_score, 'L',
                                                                                                      'D'))

In [ ]:
completed_matchup_data[['week', 'team', 'opponent', 'score', 'opponent_score', 'result', 'max_points_for', 'adjusted_result']]

## Calculate Manager Efficiency

In [ ]:
completed_matchup_data['lineup_inefficiency'] = completed_matchup_data.max_points_for - completed_matchup_data.score
completed_matchup_data['weighted_lineup_inefficiency'] = completed_matchup_data.lineup_inefficiency / completed_matchup_data.score

In [ ]:
completed_matchup_data.groupby('team')[['lineup_inefficiency']].mean().round(2).sort_values('lineup_inefficiency').rename(columns={'lineup_inefficiency': 'Average Lineup Inefficiency'})

In [ ]:
completed_matchup_data[(completed_matchup_data.week == completed_matchup_data.week.max())][['team', 'score', 'max_points_for', 'lineup_inefficiency']].sort_values('lineup_inefficiency').set_index('team')

## Calculate Optimal Standings

In [ ]:
completed_matchup_data = pd.merge(
    left = completed_matchup_data,
    right = completed_matchup_data,
    left_on = ['week', 'team', 'opponent'],
    right_on = ['week', 'opponent', 'team'],
    suffixes = [None, '_opp']
)

In [ ]:
completed_matchup_data['max_results'] = np.where(completed_matchup_data.max_points_for > completed_matchup_data.max_points_for_opp, 'W',
                                     np.where(completed_matchup_data.max_points_for < completed_matchup_data.max_points_for_opp, 'L',
                                                                                                      'D'))

In [ ]:
completed_matchup_data['max_results'].value_counts()

In [ ]:
max_standings = pd.DataFrame()
# max_standings['team_id'] = merged_matchup_data.groupby('team').team_id.last()
max_standings['wins'] = completed_matchup_data.groupby('team').max_results.agg(lambda x: x.tolist().count("W"))
max_standings['draws'] = completed_matchup_data.groupby('team').max_results.agg(lambda x: x.tolist().count("D"))
max_standings['losses'] = completed_matchup_data.groupby('team').max_results.agg(lambda x: x.tolist().count("L"))
max_standings['record'] = max_standings.wins.astype(str) + '-' + max_standings.draws.astype(str) + '-' + max_standings.losses.astype(str)
max_standings['points'] = max_standings.wins*3 + max_standings.draws
max_standings['fpts_for'] = completed_matchup_data.groupby('team').max_points_for.sum()
max_standings['fpts_against'] = completed_matchup_data.groupby('team').max_points_for_opp.sum()

In [ ]:
max_standings.sort_values(['points', 'fpts_for'], ascending=False)

## Calculate Standings Against Weekly Median

In [ ]:
weekly_medians = completed_matchup_data.groupby('week').score.median()

In [ ]:
completed_matchup_data['weekly_median_score'] = completed_matchup_data.week.apply(lambda x: weekly_medians.loc[x])

In [ ]:
completed_matchup_data['result_against_median'] = np.where(completed_matchup_data.score > completed_matchup_data.weekly_median_score, 'W', np.where(
                                                           completed_matchup_data.score < completed_matchup_data.weekly_median_score, 'L', 'D'))

In [ ]:
median_standings = pd.DataFrame()
# max_standings['team_id'] = merged_matchup_data.groupby('team').team_id.last()
median_standings['wins'] = completed_matchup_data.groupby('team').result_against_median.agg(lambda x: x.tolist().count("W"))
median_standings['draws'] = completed_matchup_data.groupby('team').result_against_median.agg(lambda x: x.tolist().count("D"))
median_standings['losses'] = completed_matchup_data.groupby('team').result_against_median.agg(lambda x: x.tolist().count("L"))
median_standings['record'] = median_standings.wins.astype(str) + '-' + median_standings.draws.astype(str) + '-' + median_standings.losses.astype(str)
median_standings['points'] = median_standings.wins*3 + median_standings.draws
median_standings['fpts_for'] = completed_matchup_data.groupby('team').score.sum()
median_standings['fpts_against'] = completed_matchup_data.groupby('team').opponent_score.sum()

In [ ]:
weekly_medians

In [ ]:
median_standings.sort_values(['points', 'fpts_for'], ascending=False)

## Analyze Data

In [ ]:
weeks = completed_matchup_data.week.max()

In [ ]:
completed_matchup_data.sort_values('score', ascending=False)

In [ ]:
for week in range(1, weeks+1):
    df = completed_matchup_data[completed_matchup_data.week == week].sort_values('lineup_inefficiency').reset_index(drop=True)
    most = df.iloc[0]
    least = df.iloc[-1]
    print('-'*20, 'Week', week, '-'*20)
    print(f'Most Efficient Manager:', most.team)
    print('\t', most.score, 'of', most.max_points_for)
    print(f'Least Efficient Manager:', least.team)
    print('\t', least.score, 'of', least.max_points_for)
    print()
    print()

In [ ]:
completed_matchups

In [ ]:
top_teams_by_week = (
    completed_matchups
        .loc[completed_matchups.groupby('week')['score'].idxmax()]
        .reset_index(drop=True)
)

In [ ]:
top_teams_by_week[['week', 'team', 'score']].set_index('week').team.value_counts()

# Data Storage

## Save

In [ ]:
for week in df_player_stats['matchweek'].unique():
    df_player_stats[df_player_stats['matchweek'] == week].to_csv(f"player_data/df_player_stats_{week}.csv", index=False)

In [ ]:
# df_player_stats.to_csv('df_player_stats.csv', index=False)
# df_players.to_csv('df_players.csv', index=False)
# completed_matchup_data.to_csv('completed_matchup_data.csv', index=False)

## Load

In [ ]:
# df_player_stats = pd.read_csv('df_player_stats.csv')
# df_players = pd.read_csv('df_players.csv')
# completed_matchup_data = pd.read_csv('completed_matchup_data.csv')

# Data Analysis

## Stats based on latest information

In [ ]:
df_player_stats.matchweek.value_counts().sort_index()

In [ ]:
# stats = df_player_stats.groupby('player_name').agg(lambda x: list(x)[-2]).sort_values(['date', 'scorerId']).reset_index()
stats = df_player_stats[(df_player_stats.matchweek == FINISHED_WEEKS) & (df_player_stats.minutes > 5)]

In [ ]:
# stats['positional_avg_fpts_per_90'] = df_player_stats.positional_avg_fpts_per_min_against_opp * 90

In [ ]:
stats['fpts_per_min_gameweek'] = stats.fpts / stats.minutes
stats['fpts_per_90_gameweek'] = stats.fpts_per_min_gameweek * 90

In [ ]:
cols = ['date', 'player_name', 'team', 'opp', 'fpts', 'minutes', 'fpts_per_min_gameweek', 'fpts_per_90_gameweek', 'fantrax_projection']

In [ ]:
stats.sort_values('fpts_per_90_gameweek', ascending=False)[cols]

# Model

## Util

In [ ]:
def join_predictions(X_test, y_test, y_pred, targets=['fpts']):
    mbd_pred = pd.concat([X_test.reset_index(drop=True), y_pred.reset_index(drop=True)], axis=1)
    mbd_pred['player_id'] = mbd_pred.player_id.apply(lambda x: player_id_ref[x])
    mbd_pred['team'] = mbd_pred.team.apply(lambda x: team_ref[x])
    mbd_pred['opp'] = mbd_pred.opp.apply(lambda x: opp_ref[x])
    mbd_pred['position'] = mbd_pred.position.apply(lambda x: position_ref[x])
    if targets == ['fpts']:
        mbd_pred['fpts_pred'] = y_pred['fpts'].tolist()
        mbd_pred['fpts_true'] = y_test['fpts'].tolist()
    else:
        mbd_pred['fpts_pred'] = mbd_pred.apply(get_fantasy_score, axis=1).round(2)
        mbd_true = pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
        mbd_true['player_id'] = mbd_true.player_id.apply(lambda x: player_id_ref[x])
        mbd_true['team'] = mbd_true.team.apply(lambda x: team_ref[x])
        mbd_true['opp'] = mbd_true.opp.apply(lambda x: opp_ref[x])
        mbd_true['position'] = mbd_true.position.apply(lambda x: position_ref[x])
        mbd_true.sort_values(['matchweek', 'player_id'], inplace=True)
        mbd_pred.sort_values(['matchweek', 'player_id'], inplace=True)
        mbd_pred['fpts_true'] = mbd_true.apply(get_fantasy_score, axis=1).round(2)

    mbd_pred['player_name'] = mbd_pred.player_id.apply(
        lambda x: df_player_stats[df_player_stats.player_id == x].player_name.values[0]
    )

    mbd_pred['error_fpts'] = mbd_pred.fpts_true - mbd_pred.fpts_pred
    mbd_pred['abs_error_fpts'] = mbd_pred.error_fpts.abs()
    mbd_pred['error_fantrax'] = mbd_pred.fpts_true - mbd_pred.fantrax_projection if 'fantrax_projection' in mbd_pred else 0
    return mbd_pred

In [ ]:
def get_team_prediction(team, mbd_pred, error=False):
    a = current_week_matchups[current_week_matchups.team == team][['g', 'd1', 'd2', 'd3', 'd4', 'd5', 'm1', 'm2', 'm3', 'm4', 'm5', 'f1', 'f2', 'f3']].T
    if not error:
        mbd_pred['fantrax_projection'] = 0
    predictions = []
    fantrax_projections = []
    fantasy_scores = []
    for position, player in a.iterrows():
        player = str(player.values[0])
        if not player == 'None':
            prediction = mbd_pred[mbd_pred.player_name == player].fpts_pred.values
            fantrax_projection = mbd_pred[mbd_pred.player_name == player].fantrax_projection.values
            fantasy_score = mbd_pred[mbd_pred.player_name == player].fpts_true.values
            if len(prediction) == 0:
                prediction = 0
            else:
                prediction = prediction[0]
            if len(fantrax_projection) == 0:
                fantrax_projection = 0
            else:
                fantrax_projection = fantrax_projection[0]
            if len(fantasy_score) == 0:
                fantasy_score = 0
            else:
                fantasy_score = fantasy_score[0]
        else:
            prediction = 0
            fantrax_projection = 0
            fantasy_score = 0
        predictions = predictions + [prediction]
        fantrax_projections = fantrax_projections + [fantrax_projection]
        fantasy_scores = fantasy_scores + [fantasy_score]
    a['predicted_fpts'] = predictions
    a['fantrax_projection'] = fantrax_projections
    a['actual_fpts'] = fantasy_scores
    
    a.columns = ['player', 'predicted_fpts', 'fantrax_projection', 'actual_fpts']
    a = a[~a.player.isna()]
    totals = pd.DataFrame({
        'player': {'total': 'Total Points'},
        'predicted_fpts': {'total': a.predicted_fpts.sum()},
        'fantrax_projection': {'total': a.fantrax_projection.sum()},
        'actual_fpts': {'total': a.actual_fpts.sum()} 
    })
    a = pd.concat([a, totals])
    if error:
        a['commish_error'] = a.predicted_fpts - a.actual_fpts
        a['fantrax_error'] = a.fantrax_projection - a.actual_fpts
    else:
        a = a.drop('actual_fpts', axis=1)
    return a

In [ ]:
def calculate_mae(errors):
    return errors.abs().mean()

In [ ]:
def calculate_mae90(errors):
    p5 = np.percentile(errors, 5)
    p95 = np.percentile(errors, 95)
    return errors[(errors > p5) & (errors < p95)].abs().mean()

## Create Model Build Data

In [ ]:
print(str(round((df_player_stats[df_player_stats.position == 'G'].shape[0] / df_player_stats.shape[0] * 100), 2)) + '%')

In [ ]:
nans = df_player_stats.isna().sum()
nans[nans > 0].div(len(df_player_stats)).multiply(100).round(2).astype(str)+'%'

In [ ]:
mbd = df_player_stats.fillna(0)

In [ ]:
mbd.columns

In [ ]:
mbd[['player_name', 'matchweek', 'team', 'opp'] + [col for col in mbd.columns if 'position' in col or 'per_90' in col]]

In [ ]:
current_week_matchups = matchup_data[matchup_data.week == CURRENT_WEEK]

## Updated Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
stats = ['g', 'kp', 'at', 'sot', 'tkw', 'dis', 'yc', 'rc', 'acnc', 'int', 'clr', 'cos', 'bs', 'aer', 'pkm', 'pkd', 'og', 'gao', 'cs', 'ga', 'sv', 'pks', 'hcs', 'sm']

In [ ]:
def join_predictions_stat(X_test, y_test, y_pred, target='fpts', mbd=mbd):
    mbd_pred = pd.concat([X_test.reset_index(drop=True), y_pred.reset_index(drop=True)], axis=1)
    mbd_pred['player_id'] = mbd_pred.player_id.apply(lambda x: player_id_ref[x])
    mbd_pred['team'] = mbd_pred.team.apply(lambda x: team_ref[x])
    mbd_pred['opp'] = mbd_pred.opp.apply(lambda x: opp_ref[x])
    mbd_pred['position'] = mbd_pred.position.apply(lambda x: position_ref[x])
    if target == 'fpts':
        mbd_pred['fpts_pred'] = y_pred['fpts'].tolist()
        mbd_pred['fpts_true'] = y_test['fpts'].tolist()
    else:
        mbd_pred[f'{target}_pred'] = mbd_pred.apply(get_fantasy_score, axis=1).round(2)
        mbd_true = pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
        mbd_true = pd.merge(mbd[['opp', 'home_ind', 'matchweek']], mbd_true, on=['opp', 'home_ind'])
        mbd_true['player_id'] = mbd_true.player_id.apply(lambda x: player_id_ref[x])
        mbd_true['team'] = mbd_true.team.apply(lambda x: team_ref[x])
        mbd_true['opp'] = mbd_true.opp.apply(lambda x: opp_ref[x])
        mbd_true['position'] = mbd_true.position.apply(lambda x: position_ref[x])
        mbd_true.sort_values(['matchweek', 'player_id'], inplace=True)
        mbd_pred.sort_values(['matchweek', 'player_id'], inplace=True)
        mbd_pred[f'{target}_true'] = mbd_true.apply(get_fantasy_score, axis=1).round(2)

    mbd_pred['player_name'] = mbd_pred.player_id.apply(
        lambda x: df_player_stats[df_player_stats.player_id == x].player_name.values[0]
    )

    mbd_pred[f'error_{target}'] = mbd_pred.fpts_true - mbd_pred.fpts_pred
    mbd_pred[f'abs_error_{target}'] = mbd_pred.error_fpts.abs()
    # mbd_pred['error_fantrax'] = mbd_pred.fpts_true - mbd_pred.fantrax_projection if 'fantrax_projection' in mbd_pred else 0
    return mbd_pred

In [ ]:
pred_stats = pd.DataFrame()
for stat in stats:
    print("#"*20, stat, "#"*20)
    features =['player_id', 'team', 'opp', 'team_week_standing', 'team_week_standing_opp', 'position', 'home_ind'] + [col for col in mbd.columns if col.startswith(f"{stat}_") or f"_{stat}_" in col and 'per_90' in col]
    target = stat
    X = mbd[features].reset_index(drop=True)
    y = mbd[[target]].reset_index(drop=True)
    
    X['player_id'], player_id_ref = pd.factorize(X.player_id)
    X['team'], team_ref = pd.factorize(X.team)
    X['opp'], opp_ref = pd.factorize(X.opp)
    X['home_ind'] = X.home_ind.astype(int)
    X['position'], position_ref = pd.factorize(X.position)
    
    # features = X.columns.tolist()
    # target = y.columns.tolist()
    
    print('features:', features)
    print()
    print('target(s):', target)
    print()
    
    train_test_split_ind = mbd[mbd.matchweek < CURRENT_WEEK].index[-1] + 1
    test_end_ind = mbd[mbd.matchweek == CURRENT_WEEK].index[-1] + 1
    
    X_train, y_train = X.iloc[:train_test_split_ind], y[:train_test_split_ind]
    X_test, y_test = X.iloc[train_test_split_ind:test_end_ind], y[train_test_split_ind:test_end_ind]
    
    print('training data shape:', X_train.shape, y_train.shape)
    print('test data shape:', X_test.shape, y_test.shape)

    # Initialize and train the model
    model = MultiOutputRegressor(RandomForestRegressor(n_estimators=300, random_state=42))
    model.fit(X_train[features], y_train[[target]])
    
    # Make predictions
    y_pred = model.predict(X_test[features])

    # y_pred = pd.DataFrame(y_pred, columns=[f"pred_{stat}"]).round(2)
    y_pred = pd.DataFrame(y_pred, columns=[stat]).round(2)
    pred_stats = pd.concat([pred_stats, y_pred], axis=1)
    # # Evaluate the model
    # for idx, target in enumerate([target]):
    #     mse = mean_squared_error(y_test[target], y_pred[:, idx])
    #     print(f"MSE for {target}: {mse}")
    #     mbd_pred_tree = join_predictions_stat(X_test, y_test, pd.DataFrame(y_pred, columns=[stat]).round(2), target=stat)

    # for team in api._teams:
    #     team = str(team)
    #     print(team)
    #     display(get_team_prediction(team, mbd_pred_tree, error=False, target=stat))
    #     print()
    print(pred_stats.shape)
    print('\n\n')

In [ ]:
pred_data = pd.concat([X_test.reset_index(drop=True), pred_stats.reset_index(drop=True)], axis=1)
pred_data['player_id'] = pred_data.player_id.apply(lambda x: player_id_ref[x])
pred_data['team'] = pred_data.team.apply(lambda x: team_ref[x])
pred_data['opp'] = pred_data.opp.apply(lambda x: opp_ref[x])
pred_data['position'] = pred_data.position.apply(lambda x: position_ref[x])

In [ ]:
pred_data['fpts_pred'] = pred_data.apply(get_fantasy_score, axis=1)

In [ ]:
pred_data['fpts_true'] = y_test

In [ ]:
pred_data['player_name'] = pred_data.player_id.apply(
        lambda x: df_player_stats[df_player_stats.player_id == x].player_name.values[0]
    )

In [ ]:
for team in api._teams:
    team = str(team)
    print(team)
    display(get_team_prediction(team, pred_data, error=False)[['player', 'predicted_fpts']])
    print()

### Error Analysis

In [ ]:
mbd_pred_tree = pd.merge(mbd[['player_id', 'opp', 'home_ind', 'matchweek', 'fpts']], pred_data, on=['player_id', 'opp', 'home_ind'])

In [ ]:
mbd_pred_tree['fpts_true'] = mbd_pred_tree.fpts

In [ ]:
mbd_pred_tree['error_fpts'] = mbd_pred_tree.fpts_pred - mbd_pred_tree.fpts_true

In [ ]:
p = mbd_pred_tree[['matchweek', 'player_name', 'position', 'opp', 'fpts_true', 'fpts_pred', 'error_fpts']]

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    errors = pd.concat([p.groupby('position').error_fpts.agg(calculate_mae)], axis=1)
    errors.columns = ['VAR Mean Absolute Error']#, 'Fantrax Mean Absolute Error']
    errors.loc[['G', 'D', 'M', 'F']]
    display(errors)

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    errors = pd.concat([p.groupby('position').error_fpts.agg(calculate_mae90)], axis=1)
    errors.columns = ['VAR Mean Absolute Error 90%']#, 'Fantrax Mean Absolute Error 90%']
    errors.loc[['G', 'D', 'M', 'F']]
    display(errors)

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    fig = px.histogram(p, 'error_fpts', color='position', nbins=60)
    fig.show()
    p['abs_error_fpts'] = p.error_fpts.abs()
    # p['abs_error_fantrax'] = p.error_fantrax.abs()
    display(p[['abs_error_fpts']].describe())

In [ ]:
def predict_outcomes(mbd_pred, current_week_matchups):
    ret = pd.DataFrame()
    for i, matchup in current_week_matchups.groupby('matchupId').first().iterrows():
        home_team = matchup.team
        away_team = matchup.opponent
        home = get_team_prediction(home_team, mbd_pred)
        away = get_team_prediction(away_team, mbd_pred)

        ret = pd.concat([
            ret,
            pd.DataFrame([{
                'Home': home_team,
                'Home Score': home.predicted_fpts.loc['total'].round(2),
                'Away Score': away.predicted_fpts.loc['total'].round(2),
                'Away': away_team,
            }])
        ], ignore_index=True)        
        print(f"{home_team}  {home.predicted_fpts.loc['total'].round(2)} - {away.predicted_fpts.loc['total'].round(2)}  {away_team}")
    return ret

In [ ]:
predict_outcomes(mbd_pred_tree, current_week_matchups)

In [ ]:
stop

## Linear Model

In [ ]:
mbd = mbd.sort_values(['matchweek', 'player_id']).reset_index(drop=True)

### Generate Train/Test Sets

In [ ]:
stats = ['g', 'kp', 'at', 'sot', 'tkw', 'dis', 'yc', 'rc', 'acnc', 'int', 'clr', 'cos', 'bs', 'aer', 'pkm', 'pkd', 'og', 'gao', 'cs', 'ga', 'sv', 'pks', 'hcs', 'sm']
targets = ['fpts'] #+ [stats]
drop = ['player_name', 'score', 'min', 'minutes', 'date', 'team_result', 'calc_fpts', 'fpts', 'apts', 'last_score', 'scorerId'] + list(set(targets + stats)) + [col for col in mbd.columns if 'per_min' in col or col.startswith('season_') or col.startswith('last_') or 'fantrax' in col]

X = mbd.drop(drop+targets, axis=1).reset_index(drop=True)
y = mbd[targets].reset_index(drop=True)

X['player_id'], player_id_ref = pd.factorize(X.player_id)
X['team'], team_ref = pd.factorize(X.team)
X['opp'], opp_ref = pd.factorize(X.opp)
X['home_ind'] = X.home_ind.astype(int)
X['position'], position_ref = pd.factorize(X.position)

features = X.columns.tolist()
targets = y.columns.tolist()

print('features:', features)
print()
print('target(s):', targets)
print()

train_test_split_ind = mbd[mbd.matchweek < CURRENT_WEEK].index[-1] + 1
test_end_ind = mbd[mbd.matchweek == CURRENT_WEEK].index[-1] + 1

X_train, y_train = X.iloc[:train_test_split_ind], y[:train_test_split_ind]
X_test, y_test = X.iloc[train_test_split_ind:test_end_ind], y[train_test_split_ind:test_end_ind]

print('training data shape:', X_train.shape, y_train.shape)
print('test data shape:', X_test.shape, y_test.shape)

### Train Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
regr = LinearRegression().fit(X_train, y_train)

In [ ]:
y_pred = pd.DataFrame(regr.predict(X_test)).round(2)

In [ ]:
y_pred.columns = y_test.columns

In [ ]:
mbd_pred_lin = join_predictions(X_test, y_test, y_pred)

### Error Analysis

In [ ]:
for team in api._teams:
    team = str(team)
    print(team)
    display(get_team_prediction(team, mbd_pred_lin, error=False))
    print()

In [ ]:
p = mbd_pred_lin[['matchweek', 'player_name', 'position', 'opp', 'fpts_true', 'fpts_pred', 'error_fpts', 'error_fantrax', 'fantrax_projection']]

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    errors = pd.concat([p.groupby('position').error_fpts.agg(calculate_mae), p.groupby('position').error_fantrax.agg(calculate_mae)], axis=1)
    errors.columns = ['VAR Mean Absolute Error', 'Fantrax Mean Absolute Error']
    errors.loc[['G', 'D', 'M', 'F']]
    display(errors)

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    errors = pd.concat([p.groupby('position').error_fpts.agg(calculate_mae90), p.groupby('position').error_fantrax.agg(calculate_mae90)], axis=1)
    errors.columns = ['VAR Mean Absolute Error 90%', 'Fantrax Mean Absolute Error 90%']
    errors.loc[['G', 'D', 'M', 'F']]
    display(errors)

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    display(p.error_fpts.plot.hist(bins=50))
    p['abs_error_fpts'] = p.error_fpts.abs()
    p['abs_error_fantrax'] = p.error_fantrax.abs()
    display(p[['abs_error_fpts', 'abs_error_fantrax']].describe())

In [ ]:
predict_outcomes(mbd_pred_lin, current_week_matchups)

## Neural Network

### Generate Train/Test Sets

In [ ]:
stats = ['g', 'kp', 'at', 'sot', 'tkw', 'dis', 'yc', 'rc', 'acnc', 'int', 'clr', 'cos', 'bs', 'aer', 'pkm', 'pkd', 'og', 'gao', 'cs', 'ga', 'sv', 'pks', 'hcs', 'sm']
targets = ['fpts'] #+ [stats]
drop = ['player_name', 'score', 'min', 'minutes', 'date', 'team_result', 'calc_fpts', 'fpts', 'apts', 'last_score', 'scorerId'] + list(set(targets + stats)) + [col for col in mbd.columns if 'per_min' in col or col.startswith('season_') or col.startswith('last_') or 'fantrax' in col]

X = mbd.drop(drop+targets, axis=1).reset_index(drop=True)
y = mbd[targets].reset_index(drop=True)

X['player_id'], player_id_ref = pd.factorize(X.player_id)
X['team'], team_ref = pd.factorize(X.team)
X['opp'], opp_ref = pd.factorize(X.opp)
X['home_ind'] = X.home_ind.astype(int)
X['position'], position_ref = pd.factorize(X.position)

features = X.columns.tolist()
targets = y.columns.tolist()

print('features:', features)
print()
print('target(s):', targets)
print()

train_test_split_ind = X[X.matchweek < CURRENT_WEEK - 1].index[-1] + 1
test_val_split_ind = X[X.matchweek < CURRENT_WEEK].index[-1] + 1
test_end_ind = X[X.matchweek == CURRENT_WEEK].index[-1] + 1

X_train, y_train = X.iloc[:train_test_split_ind], y[:train_test_split_ind]
X_val, y_val = X.iloc[train_test_split_ind:test_val_split_ind], y[train_test_split_ind:test_val_split_ind]
X_test, y_test = X.iloc[test_val_split_ind:test_end_ind], y[test_val_split_ind:test_end_ind]

print('training data shape:', X_train.shape, y_train.shape)
print('validation data shape:', X_val.shape, y_val.shape)
print('test data shape:', X_test.shape, y_test.shape)

### Train Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Step 1: Convert data into PyTorch tensors
# Assume X_train, X_test, y_train, y_test are Pandas DataFrames
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.astype(float).values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test[features].values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.astype(float).values, dtype=torch.float32)

In [ ]:
# Step 2: Create DataLoader for batching
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Step 3: Define the Neural Network
class PlayerPerformanceNN(nn.Module):
    def __init__(self, input_size, hidden1, hidden2, output_size):
        super(PlayerPerformanceNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1)  # Input to first hidden layer
        self.relu1 = nn.ReLU()                    # Activation function
        self.fc2 = nn.Linear(hidden1, hidden2)    # First to second hidden layer
        self.relu2 = nn.ReLU()                    # Activation function
        self.fc3 = nn.Linear(hidden2, output_size)  # Output layer
    
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return self.fc3(x)

In [ ]:
# Model parameters
input_size = len(features)  # Number of input features
hidden1, hidden2 = 64, 32  # Neurons in hidden layers
output_size = y_train.shape[1] if len(y_train.shape) > 1 else 1  # Targets

In [ ]:
# Initialize the model
model = PlayerPerformanceNN(input_size, hidden1, hidden2, output_size)

In [ ]:
# Step 4: Define Loss and Optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [ ]:
# Step 5: Training Loop
epochs = 250
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()  # Clear gradients from the last step
        outputs = model(batch_X)  # Forward pass
        loss = criterion(outputs, batch_y)  # Compute the loss
        loss.backward()  # Backpropagate the error
        optimizer.step()  # Update weights
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader)}")


In [ ]:
# Step 6: Evaluate the Model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    y_pred = model(X_test_tensor)  # Predict on test data
    test_loss = criterion(y_pred, y_test_tensor)  # Compute test loss
print(f"Test Loss: {test_loss.item()}")

In [ ]:
# Step 7: Access Predictions
y_pred = pd.DataFrame(y_pred.numpy(), columns=y_test.columns).round(2).astype(float)

In [ ]:
mbd_pred_nn = join_predictions(X_test, y_test, y_pred, targets)

### Error Analysis

In [ ]:
for team in api._teams:
    team = str(team)
    print(team)
    display(get_team_prediction(team, mbd_pred_nn, error=False))
    print()

In [ ]:
p = mbd_pred_nn[['matchweek', 'player_name', 'position', 'opp', 'fpts_true', 'fpts_pred', 'error_fpts', 'error_fantrax', 'fantrax_projection']]

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    errors = pd.concat([p.groupby('position').error_fpts.agg(calculate_mae), p.groupby('position').error_fantrax.agg(calculate_mae)], axis=1)
    errors.columns = ['VAR Mean Absolute Error', 'Fantrax Mean Absolute Error']
    errors.loc[['G', 'D', 'M', 'F']]
    display(errors)

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    errors = pd.concat([p.groupby('position').error_fpts.agg(calculate_mae90), p.groupby('position').error_fantrax.agg(calculate_mae90)], axis=1)
    errors.columns = ['VAR Mean Absolute Error 90%', 'Fantrax Mean Absolute Error 90%']
    errors.loc[['G', 'D', 'M', 'F']]
    display(errors)

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    display(p.error_fpts.plot.hist(bins=50))
    p['abs_error_fpts'] = p.error_fpts.abs()
    p['abs_error_fantrax'] = p.error_fantrax.abs()
    display(p[['abs_error_fpts', 'abs_error_fantrax']].describe())

In [ ]:
predict_outcomes(mbd_pred_nn, current_week_matchups)

## Decision Tree Model

### Generate Train/Test Sets

In [ ]:
stats = ['g', 'kp', 'at', 'sot', 'tkw', 'dis', 'yc', 'rc', 'acnc', 'int', 'clr', 'cos', 'bs', 'aer', 'pkm', 'pkd', 'og', 'gao', 'cs', 'ga', 'sv', 'pks', 'hcs', 'sm']
targets = ['fpts'] #+ [stats]
drop = ['player_name', 'score', 'min', 'minutes', 'date', 'team_result', 'calc_fpts', 'fpts', 'apts', 'last_score', 'scorerId'] + list(set(targets + stats)) + [col for col in mbd.columns if 'per_min' in col or col.startswith('season_') or col.startswith('last_') or 'fantrax' in col]

X = mbd.drop(drop+targets, axis=1).reset_index(drop=True)
y = mbd[targets].reset_index(drop=True)

X['player_id'], player_id_ref = pd.factorize(X.player_id)
X['team'], team_ref = pd.factorize(X.team)
X['opp'], opp_ref = pd.factorize(X.opp)
X['home_ind'] = X.home_ind.astype(int)
X['position'], position_ref = pd.factorize(X.position)

features = X.columns.tolist()
targets = y.columns.tolist()

print('features:', features)
print()
print('target(s):', targets)
print()

train_test_split_ind = mbd[mbd.matchweek < CURRENT_WEEK].index[-1] + 1
test_end_ind = mbd[mbd.matchweek == CURRENT_WEEK].index[-1] + 1

X_train, y_train = X.iloc[:train_test_split_ind], y[:train_test_split_ind]
X_test, y_test = X.iloc[train_test_split_ind:test_end_ind], y[train_test_split_ind:test_end_ind]

print('training data shape:', X_train.shape, y_train.shape)
print('test data shape:', X_test.shape, y_test.shape)

### Train Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# Initialize and train the model
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=300, random_state=42))
model.fit(X_train[features], y_train[targets])

# Make predictions
y_pred = model.predict(X_test[features])

# Evaluate the model
for idx, target in enumerate(targets):
    mse = mean_squared_error(y_test[target], y_pred[:, idx])
    print(f"MSE for {target}: {mse}")

In [ ]:
mbd_pred_tree = join_predictions(X_test, y_test, pd.DataFrame(y_pred, columns=['fpts']).round(2))

### Error Analysis

In [ ]:
for team in api._teams:
    team = str(team)
    print(team)
    display(get_team_prediction(team, mbd_pred_tree, error=False))
    print()

In [ ]:
p = mbd_pred_tree[['matchweek', 'player_name', 'position', 'opp', 'fpts_true', 'fpts_pred', 'error_fpts', 'error_fantrax', 'fantrax_projection']]

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    errors = pd.concat([p.groupby('position').error_fpts.agg(calculate_mae), p.groupby('position').error_fantrax.agg(calculate_mae)], axis=1)
    errors.columns = ['VAR Mean Absolute Error', 'Fantrax Mean Absolute Error']
    errors.loc[['G', 'D', 'M', 'F']]
    display(errors)

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    errors = pd.concat([p.groupby('position').error_fpts.agg(calculate_mae90), p.groupby('position').error_fantrax.agg(calculate_mae90)], axis=1)
    errors.columns = ['VAR Mean Absolute Error 90%', 'Fantrax Mean Absolute Error 90%']
    errors.loc[['G', 'D', 'M', 'F']]
    display(errors)

In [ ]:
if CURRENT_WEEK == FINISHED_WEEKS:
    display(p.error_fpts.plot.hist(bins=50))
    p['abs_error_fpts'] = p.error_fpts.abs()
    p['abs_error_fantrax'] = p.error_fantrax.abs()
    display(p[['abs_error_fpts', 'abs_error_fantrax']].describe())

In [ ]:
predict_outcomes(mbd_pred_tree, current_week_matchups)

# Waivers Analysis

## Get Waivers

In [ ]:
df_player_stats['games_played'] = df_player_stats.groupby('player_name').matchweek.transform(lambda x: x.count() - 1)

In [ ]:
players_this_week = matchup_data[matchup_data.week == CURRENT_WEEK][['g', 'd1', 'd2', 'd3',
       'd4', 'd5', 'm1', 'm2', 'm3', 'm4', 'm5', 'f1', 'f2', 'f3', 'res1',
       'res2', 'res3', 'res4', 'res5', 'inj']].values.flatten()

players_this_week = [str(player) for player in players_this_week if player]

In [ ]:
cols = ['matchweek', 'date', 'player_name', 'team', 'position', 'opp', 'score', 'fpts', 'minutes', 'last_fpts',
       'last_minutes', 'season_fpts', 'season_minutes',
       'fpts_per_min_gameweek', 'fpts_per_min_season', 'fpts_per_90_gameweek',
       'fpts_per_90_season', 'games_played']

In [ ]:
waivers = df_player_stats[(~df_player_stats.player_name.isin(players_this_week)) & (df_player_stats.matchweek == CURRENT_WEEK)][cols]

In [ ]:
waivers[waivers.position!='G'].sort_values('season_minutes', ascending=False)

## Waiver Evaluations

### Players who have played for more than 45 minutes last week, ordered by their performance per minute last week

In [ ]:
waivers[
(waivers.position!='G') &
(waivers.last_minutes > 45) &
(waivers.matchweek == CURRENT_WEEK)
].sort_values('fpts_per_min_gameweek', ascending=False)

### Players who have played for more than half of the season, ordered by their performance per minute all season

In [ ]:
waivers[
(waivers.position!='G') &
(waivers.season_minutes > (45 * (CURRENT_WEEK)))
].sort_values('fpts_per_min_season', ascending=False)

### Players who have played a decent amount of minutes (regular subs and irregular starters), ordered by their performance per minute all season

In [ ]:
waivers[
(waivers.position!='G') &
(waivers.season_minutes > (25 * (CURRENT_WEEK))) &
(waivers.matchweek == CURRENT_WEEK)
].sort_values('fpts_per_min_season', ascending=False).head(25)

### Average fpts per game

In [ ]:
waivers['avg_fpts_per_game'] = waivers['season_fpts'] / waivers['games_played']

### How does a certain player fare when playing against teams by their rank

In [ ]:
players_of_interest = ['Emiliano Buendia', 'Bertrand Traore', 'Diego Gómez', 'Ryan Sessegnon']

In [ ]:
df_poi = pd.DataFrame()
df_opp_ranks = pd.DataFrame()
for player_of_interest in players_of_interest:
    df_opp_rank = pd.DataFrame()
    df = df_player_stats[df_player_stats.player_name.str.contains(player_of_interest)]
    p = df.player_name.values[0]
    df_poi = pd.concat([df_poi, df[df.matchweek == FINISHED_WEEKS + 1][['player_name', 'team', 'opp', 'team_week_standing_opp']].set_index('player_name')])

    df = df[df.minutes > 0]
    df = df.groupby('team_week_standing_opp').fpts.agg(list).reset_index().set_index('team_week_standing_opp')
    df.columns = [p]
    df_opp_ranks = df_opp_ranks.join(df, how='outer').fillna('N/A')

df_poi.columns = ['Team', f'Week {FINISHED_WEEKS + 1} Opponent', 'Opponent Rank']
display(df_poi)
display(df_opp_ranks)

# Player Data Analysis

In [ ]:
significant_time_played = df_player_stats[(df_player_stats.matchweek == CURRENT_WEEK) & (df_player_stats.season_minutes >= CURRENT_WEEK * 90 / 5)].player_name.tolist()

In [ ]:
data = df_player_stats[(df_player_stats['minutes'] >= 45) & (df_player_stats.player_name.isin(significant_time_played))].groupby('player_name').fpts.mean().sort_values(ascending=False).reset_index()

In [ ]:
teams = matchup_data[matchup_data.week == 17][['team', 'g', 'd1', 'd2', 'd3', 'd4', 'd5', 'm1', 'm2', 'm3', 'm4', 'm5', 'f1', 'f2', 'f3', 'res1', 'res2', 'res3', 'res4', 'res5', 'inj']].values

In [ ]:
team_dict = dict()
for team in teams.tolist():
    team_dict[team[0]] = team[1:]

In [ ]:
player_to_team = {
    player.name: team
    for team, roster in team_dict.items()
    for player in roster if player and not type(player) == float
}

In [ ]:
data['team'] = data.player_name.apply(lambda x: player_to_team[x] if x in player_to_team.keys() else 'Waivers')

In [ ]:
data[['team', 'player_name', 'fpts']].head(50)

In [ ]:
data[['team', 'player_name', 'fpts']].head(50).groupby('team').player_name.count().sort_values(ascending=False)

In [ ]:
cols_of_interest = ['g', 'kp', 'at', 'sot', 'tkw', 'dis', 'acnc', 'int', 'clr', 'cos', 'bs', 'aer', 'gao', 'cs', 'sv', 'hcs', 'sm', 'minutes']

In [ ]:
total_stats = df_player_stats[df_player_stats.matchweek < FINISHED_WEEKS].groupby('player_name')[['matchweek'] + cols_of_interest].sum().reset_index()

In [ ]:
total_stats

In [ ]:
waiver_stats = total_stats[~total_stats.player_name.isin(players_this_week)]

In [ ]:
for col in cols_of_interest:
    waiver_stats[col+'_per_min'] = waiver_stats[col] / waiver_stats['minutes']

In [ ]:
for col in cols_of_interest:
    if col == 'minutes':
        continue
    print(col.upper())
    display(waiver_stats[waiver_stats.minutes > 450].sort_values(f'{col}_per_min', ascending=False)[['player_name', col, 'minutes',f'{col}_per_min']].reset_index(drop=True).head(8))
    print()
    print()
    print()

In [ ]:
waiver_stats

In [ ]:
waiver_stats.matchweek.value_counts().sort_index()

In [ ]:
mat = df_player_stats[(df_player_stats.matchweek.isin(list(range(9, 17)))) & (df_player_stats.player_name.str.contains('Mateus')) & (df_player_stats.team == 'WHU')]

In [ ]:
top_20 = df_player_stats[(df_player_stats.matchweek.isin(list(range(9, 17)))) & (df_player_stats.position == 'M')].groupby('player_name').fpts.mean().sort_values(ascending=False).head(20)

In [ ]:
top_20

In [ ]:
df_player_stats[(df_player_stats.player_name.isin(top_20.index.tolist())) & (df_player_stats.matchweek.isin(list(range(9, 17)))) & (df_player_stats.position == 'M')].groupby('player_name').fpts.std().sort_values().head(20)